# Scratchbook

In [20]:
from tika import parser
import pandas as pd
import numpy as np

In [21]:
art_df = pd.DataFrame()

In [22]:
file = 'Bon Picnic 2023-10-11.pdf'

In [23]:
parsedPDF = parser.from_file(file)
content = parsedPDF['content']
contentlist = content.split('\n')
contentlist = list(filter(lambda a: a != '', contentlist))
contentlist = list(filter(lambda a: a != ' ', contentlist))

In [24]:
def find_int(entry):
    try:
        out = isinstance(int(entry), int)
    except:
        out = False
    return out
t = pd.DataFrame(contentlist).apply(lambda x: find_int(x), axis=1)

C:\Users\G23236\AppData\Local\Temp\ipykernel_17868\4004558267.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  out = isinstance(int(entry), int)


Einzelne Positionen

In [25]:
preis_list = []
reduziert_list = []
original_preis = []
menge_list = []
artikel_list = []
gebinde_list = []
for position in pd.Series(t[t].index):
    artikel_list.append(contentlist[position+1])
    menge_list.append(int(contentlist[position]))
    text = contentlist[position+2]
    end = text.rfind(' ')
    if end < 0:
        end = len(text)
    gebinde_list.append(text[:end])
    if contentlist[position+3][0] == 'j': #reduzierter Preis
        preis_list.append(float(contentlist[position+3][6:-1]))
        reduziert_list.append(True)
        original_preis.append(float(contentlist[position+4]))
    else:
        preis_list.append(float(contentlist[position+2].split(' ')[-1]))
        reduziert_list.append(False)
        original_preis.append(float(contentlist[position+2].split(' ')[-1]))
art_df['position'] = range(1,len(pd.Series(t[t].index))+1)
art_df['Artikel'] = artikel_list
einzelpreis_list = [i / j for i, j in zip(preis_list, menge_list)]
art_df['Einzelpreis reduziert'] = einzelpreis_list
art_df['Menge'] = menge_list
art_df['Gesamtpreis reduziert'] = preis_list
art_df['Reduziert'] = reduziert_list
einzelpreis_original_list = [i / j for i, j in zip(original_preis, menge_list)]
art_df['Einzelpreis original'] = einzelpreis_original_list
art_df['Gesamtpreis original'] = original_preis
unit_list = []
for gebinde in gebinde_list:
    unit_list.append(gebinde[[(i, c) for i,c in enumerate(gebinde) if c.isdigit()][-1][0]+1:].strip())
measure_list = []
for gebinde in gebinde_list:
    meas = gebinde[:[(i, c) for i,c in enumerate(gebinde) if c.isdigit()][-1][0]+1]
    mult =  meas.find(' x ')
    if mult > 0:
        meas = float(float(meas[:mult])*float(meas[mult+3:]))
    else:
        meas = float(meas)
    measure_list.append(meas)
art_df['Gebinde'] = gebinde_list
art_df['quantity'] = measure_list
art_df['unit'] = unit_list

ValueError: could not convert string to float: '180g'

In [31]:
contentlist

['Hallo Georg,',
 'hier ist der Bon zu deiner Bestellung vom Samstag 23.',
 'September. Klick hier, um ihn herunterzuladen oder zu',
 'drucken.',
 'Bestellt am Donnerstag 21. September',
 'Bestellnr 202-340-6883',
 '1',
 'Backpapier Zuschnitte',
 '30 St. 1.20',
 '2',
 'The Vegetarian Butcher Veganes Crispy Chicken',
 '180g',
 '2 für 4.99€',
 '6.38',
 '4.99',
 '2',
 'Maggi Buchstaben Suppe',
 '100g .00',
 '1',
 'Maggi Grießklößchen Suppe',
 '50g .00',
 '1',
 'Maggi Hochzeits Suppe',
 '55g',
 '3+1 gratis',
 '3.96',
 '2.97',
 '1',
 'Salatgurke',
 '1 St. mind. 300g 0.69',
 '1',
 'Eisbergsalat',
 '1 St. 1.19',
 '1',
 'Cheddar 50%',
 '175g 2.19',
 '1',
 'Craft Burger Buns vegan',
 '4 St. 1.99',
 '1',
 'Remoulade',
 '500ml 1.85',
 '1',
 'Backofen Frites extra dünn',
 '750g 2.29',
 '1',
 'Radieschen',
 '1 Bund 0.79',
 '1',
 'Himbeeren',
 '125g 2.49',
 '1',
 'Mini Pflaumen Tomaten',
 '250g',
 'jetzt 1.89€',
 '2.49',
 '1.89',
 '1',
 'Avocado Hass',
 '2 St. 1.00',
 'http://email.picnic.de/optiext

Lieferdatum

In [ ]:
end = ''.join(contentlist[0:pd.Series(t[t].index)[0]-2]).find('Klick')
txt = ''.join(contentlist[0:pd.Series(t[t].index)[0]-2])[:end-2]
start = txt.find('.')
month = txt[start+1:]
day = txt[:start][-2:].strip()
art_df['delivery_day'] = day
art_df['delivery_month'] = month

DeliveryID

In [ ]:
art_df['deliveryID'] = file

Bestellnummer

In [ ]:
t2 = pd.DataFrame(contentlist).apply(lambda x: str(x)[5:14]=='Bestellnr', axis=1)

In [ ]:
bestnr_list = []
for bestnr in pd.Series(t2[t2].index):
    space = contentlist[bestnr].find(' ')
    bestnr_list.append(contentlist[bestnr][space+1:])
bestnr_list

['903-680-6242', '904-810-6093', '904-780-6212']

In [ ]:
idx_artikel = pd.Series(t[t].index)
idx_bestellnr = pd.Series(t2[t2].index)

In [ ]:
bestell_list = []
k = -1
for i in range(len(idx_artikel)):
    if k < len(idx_bestellnr)-1 and i < len(idx_artikel)-1:
        if idx_artikel[i] > idx_bestellnr[k+1]:
            k += 1
    bestell_list.append(bestnr_list[k])
art_df['order-nr'] = bestell_list

Hinzugefügt am

In [ ]:
add_month_list = []
add_day_list = []
for idx in idx_bestellnr-1:
    dot = contentlist[idx].find('.')
    add_month_list.append(contentlist[idx][dot+2:])
    add_day_list.append(contentlist[idx][:dot][-2:].strip())
add_df = pd.DataFrame({'order-nr.': bestnr_list, 'add_day': add_day_list, 'add_month': add_month_list})

In [ ]:
art_df = art_df.join(add_df.set_index('order-nr.'), on=('order-nr'))

In [ ]:
art_df

,position,Artikel,Einzelpreis reduziert,Menge,Gesamtpreis reduziert,Reduziert,Einzelpreis original,Gesamtpreis original,Gebinde,quantity,unit,delivery_day,delivery_month,deliveryID,order-nr,add_day,add_month
0,1,Mandarinen,1.49,1,1.49,True,2.99,2.99,750g,750.0,g,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
1,2,Rote Johannisbeeren,2.22,1,2.22,True,2.49,2.49,200g,200.0,g,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
2,3,Schmücker Hof Erdbeeren,3.69,1,3.69,True,3.99,3.99,400g,400.0,g,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
3,4,Herta Finesse Hähnchenbrust gegrillt,1.79,1,1.79,True,2.49,2.49,100g,100.0,g,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
4,5,Lätta extra Fit,1.49,1,1.49,True,1.99,1.99,450g,450.0,g,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
5,6,Duschdas Duschgel Zitrone & Rosmarin,1.29,1,1.29,True,1.45,1.45,225ml,225.0,ml,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
6,7,"Frische Milch 3,5%",0.99,2,1.98,False,0.99,1.98,1L,1.0,L,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
7,8,Bananen,1.27,1,1.27,False,1.27,1.27,5 St.,5.0,St.,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
8,9,Tante Fanny Flammkuchenteig,1.99,1,1.99,False,1.99,1.99,260g,260.0,g,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September
9,10,Lauchzwiebeln,0.99,1,0.99,False,0.99,0.99,1 Bund,1.0,Bund,26,September,Bon Picnic 2023-09-26.pdf,903-680-6242,25,September


Pfand (auf Testfall warten)
Über alle Bons
Kategorien (ChatGPT)

In [ ]:
txt = ''.join(contentlist)
pos = txt.find('Eingereichtes Pfand')
if pos < 0:
    pfand = 0
else:
    #Pfand ermitteln

SyntaxError: incomplete input (876034596.py, line 6)